WARNING: experimental FSI + .net standard

In [ ]:
#load "Paket.fsx"

In [ ]:
Paket.Version [ 
    "Microsoft.ML", "0.5"
    "NETStandard.Library", "2.0.3"]

In [ ]:
//Patch location for native dependencies
System.Environment.SetEnvironmentVariable("Path", 
    System.Environment.GetEnvironmentVariable("Path") + ";" + Paket.RootPath + @"/packages/Microsoft.ML/runtimes/win-x64/native/")
    
System.Environment.SetEnvironmentVariable("Path", 
    System.Environment.GetEnvironmentVariable("Path") + ";" + Paket.RootPath + @"/packages/Microsoft.ML.CpuMath/runtimes/win-x64/native/")

In [ ]:
#r "packages/NETStandard.Library/build/netstandard2.0/ref/netstandard"
#load "Paket.Generated.Refs.fsx"

In [ ]:
open Microsoft.ML
open Microsoft.ML.Runtime.Api
open Microsoft.ML.Transforms
open Microsoft.ML.Trainers
open System
open Microsoft.ML.Data

In [ ]:
Util.Url "https://github.com/dotnet/machinelearning/blob/master/test/data/wikipedia-detox-250-line-data.tsv"

In [ ]:
let testDataPath = @"wikipedia-detox-250-line-data.tsv"

In [ ]:
type SentimentData() =
    [<Column(ordinal = "0", name = "Label"); DefaultValue>]
    val mutable Sentiment : float32
    [<Column(ordinal = "1"); DefaultValue>]
    val mutable SentimentText : string

type SentimentPrediction() =
    [<ColumnName "PredictedLabel"; DefaultValue>]
    val mutable Sentiment : bool

In [ ]:
let pipeline = LearningPipeline()
pipeline.Add(TextLoader(testDataPath).CreateFrom<SentimentData>())
pipeline.Add(TextFeaturizer("Features", "SentimentText"))
pipeline.Add(FastTreeBinaryClassifier(NumLeaves = 5, NumTrees = 5, MinDocumentsInLeafs = 2))

In [ ]:
let model = pipeline.Train<SentimentData, SentimentPrediction>()

In [ ]:
//let evaluator = new BinaryClassificationEvaluator();

In [ ]:
let predictions =
    [ SentimentData(SentimentText = "Contoso's 11 is a wonderful experience")
      SentimentData(SentimentText = "Oooooh thank you Mr. DietLimeCola. Once again, nice job trying to pretend you have some authority over anybody here. You are a wannabe admin, which is even sadder than a real admin") ]
    |> List.map model.Predict

predictions
|> Seq.iter(fun p -> printfn "%b" p.Sentiment)

In [ ]:
let predictions =
    [ SentimentData(SentimentText = "SOMETHING AWFUL IS DEAD DEAD DEAD DEAD DEAD")]
    |> List.map model.Predict

predictions
|> Seq.iter(fun p -> printfn "%b" p.Sentiment)